In [1]:
%cd crosscoder_fun

[Errno 2] No such file or directory: 'crosscoder_fun'
/crosscoder_fun


In [ ]:
# %%
import os
import sys
sys.path.append('/crosscoder_fun')

from utils import *
from crosscoder import CrossCoder
torch.set_grad_enabled(False);

In [5]:

# %%
cross_coder = CrossCoder.load("version_0",0)

# %%
norms = cross_coder.W_dec.norm(dim=-1)
norms.shape
# %%
relative_norms = norms[:, 1] / norms.sum(dim=-1)
relative_norms.shape
# %%

fig = px.histogram(
    relative_norms.detach().cpu().numpy(), 
    title="Pythia 160m de duped crosscoder checkpoint 140000 v. finished model<br> <sub>Left is finished, right almost finished</sub>",
    labels={"value": "Relative decoder norm strength"},
    nbins=200,
)

fig.update_layout(showlegend=False)
fig.update_yaxes(title_text="Number of Latents")

# Update x-axis ticks
fig.update_xaxes(
    tickvals=[0, 0.25, 0.5, 0.75, 1.0],
    ticktext=['0', '0.25', '0.5', '0.75', '1.0']
)

fig.show()

{'batch_size': 4096,
 'beta1': 0.9,
 'beta2': 0.999,
 'buffer_mult': 180,
 'd_in': 768,
 'dec_init_norm': 0.08,
 'device': 'cuda:0',
 'dict_size': 12288,
 'enc_dtype': 'fp32',
 'hook_point': 'blocks.5.hook_resid_pre',
 'l1_coeff': 2,
 'log_every': 20,
 'lr': 5e-05,
 'model_batch_size': 32,
 'model_name': 'pythia-160m-deduped',
 'num_tokens': 25000000,
 'save_every': 30000,
 'seed': 49,
 'seq_len': 256,
 'site': 'resid_pre',
 'wandb_project': 'crosscoder-actual',
 'wandb_run_name': 'l1_2_25_m_toks_base_and_almost_done'}


/crosscoder_fun/crosscoder.py:212: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



In [6]:
fig.write_html('near_fig.html')

In [5]:
# %%
from utils import *
from trainer import Trainer
# %%
device = 'cuda:0'

base_model = HookedTransformer.from_pretrained(
    "pythia-160m-deduped", 
    device=device, 
)

checkpoint_mid_model = HookedTransformer.from_pretrained(
    "EleutherAI/pythia-160m-deduped", 
    device=device, 
    checkpoint_value = 512
)

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


Loaded pretrained model pythia-160m-deduped into HookedTransformer
Loaded pretrained model EleutherAI/pythia-160m-deduped into HookedTransformer


In [6]:

import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from tqdm import tqdm
import os
from pathlib import Path
def compile_all_tokens(sequence_length=256, batch_size=512, max_batches=1000, device='cuda:0', save_dir=None):
    """
    Iterates through the pile_dedup_sample dataset and compiles a new all_tokens tensor.
    If a saved tensor exists at save_dir, loads that instead of recomputing.

    Args:
        sequence_length (int): The length of each token sequence.
        batch_size (int): The number of sequences per batch.
        max_batches (int, optional): The maximum number of batches to process. Defaults to None.
        device (str): The device to store the tensor on.
        save_dir (str, optional): Directory to save/load tensor from. If None, uses default name.

    Returns:
        torch.Tensor: A tensor containing all tokenized sequences.
    """
    # Create default save path if none provided
    if save_dir is None:
        save_dir = f"all_tokens_seq{sequence_length}_batch{batch_size}_max{max_batches}.pt"
    save_path = Path(save_dir)

    # Check if tensor already exists
    if save_path.exists():
        print(f"Loading existing tensor from {save_path}")
        return torch.load(save_path).to(device)

    # Load dataset
    pile_dedup_sample = load_dataset('EleutherAI/the_pile_deduplicated', streaming=True, split='train')

    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained('EleutherAI/pythia-160m-deduped')

    all_token_batches = []

    current_batch = []
    batch_count = 0

    print("Compiling all_tokens tensor...")
    pbar = tqdm(total=max_batches)
    for sample in pile_dedup_sample:
        # Tokenize the text
        tokens = tokenizer.encode(sample['text'])
        
        # Get as many sequence_length chunks as possible from tokens
        for i in range(0, len(tokens) - sequence_length + 1, sequence_length):
            current_batch.append(tokens[i:i + sequence_length])
            
            # If batch is full, add to all_token_batches
            if len(current_batch) == batch_size:
                batch_tensor = torch.tensor(current_batch, dtype=torch.int32)
                all_token_batches.append(batch_tensor)
                current_batch = []
                batch_count += 1
                pbar.update(1)
                # Check if we've reached the maximum number of batches
                if max_batches and batch_count >= max_batches:
                    break
        
        # Break outer loop if we've hit max_batches
        if max_batches and batch_count >= max_batches:
            break

    pbar.close()

    # Handle the last batch if it's not empty
    if current_batch:
        batch_tensor = torch.tensor(current_batch, dtype=torch.int32)
        all_token_batches.append(batch_tensor)

    # Concatenate all batches into a single tensor
    all_tokens = torch.cat(all_token_batches, dim=0).to(device)

    # Save tensor
    print(f"Saving tensor to {save_path}")
    save_path.parent.mkdir(parents=True, exist_ok=True)
    torch.save(all_tokens.cpu(), save_path)

    print(f"Compiled all_tokens tensor with shape: {all_tokens.shape}")
    return all_tokens


In [7]:
actual_toks = compile_all_tokens()

Loading existing tensor from all_tokens_seq256_batch512_max1000.pt


/tmp/ipykernel_6151/1812725462.py:30: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



In [22]:
#manually add a bos token 0 to the start of each sequence
actual_toks2 = torch.cat([torch.zeros(actual_toks.shape[0], 1, device=actual_toks.device, dtype=torch.int32), actual_toks[:, :-1]], dim=1)
actual_toks2.shape

torch.Size([512000, 256])

In [23]:
actual_toks2[0:10,0:10]

tensor([[    0,  1147,   310,  2218,    13,   285,  9262,    15,  1422,   476],
        [    0,  4566,  1411,   752,  5606,   310,  6326,   281,   320,   751],
        [    0, 47190,  2820,   281,  3523,   731,   432,  2509,   594,    15],
        [    0,  1679,   715,   436,    27,   368,   403,  7063,   247,  2829],
        [    0,  1475,    15,  1310, 27725,   432,   643,  2285,   403,  1475],
        [    0,   812,  1056,   247,  4522,   293,  8023,  3492,  1996,    15],
        [    0,  2216,    13,   309,  7402,   598,  1469,   342,  8489, 15958],
        [    0,   187,   688,   253,   990,    13,   253,  2934,   273,  1469],
        [    0,    13,   309,  7402,   598, 47348,   954,   273,   731,    13],
        [    0,   273,  1841,   497,   417,  2218,   187,   187,  6014,   780]],
       device='cuda:0', dtype=torch.int32)

In [10]:
actual_toks.shape

torch.Size([512000, 256])

In [13]:
actual_toks[1]

tensor([ 4566,  1411,   752,  5606,   310,  6326,   281,   320,   751,    15,
         1310,   368,  1581,   253,  2608,   281,  2619,   849,   281, 23554,
         1633,    13,   352,   457,    84,   417,  5606, 10542,  1108,   352,
          457,    84,   253,  6425,   273, 17497,  2216,    13,   253,   269,
          494, 23179,   407,  8869,  1346,   281, 11757,   253,  1077,  2934,
          273,  5606,    15, 16688,   639,    79,  6932,   285,   247, 20840,
         2320,  6656,    13,   326,   457,    84,   417,  1633,   326, 32089,
          479,   253,   987,  1039,    15,   187,   187, 35914,    13,   619,
         5962,   277,  4002,   785,   672, 18000,   752,   281,  2794,   369,
          417,   342,   752,   309,  3078,   281,  2794,    13,   533,   342,
          752,   309,   858,   417,    15,   309,  1904,   457,    85,   971,
          281,  2794,   271,   773,   565, 46207,  2216,   668, 40022,   285,
        47723,  1067,   352,  5606,    15,   187,   187,  1552, 

In [14]:
def compile_all_tokens2(sequence_length=256, batch_size=512, max_batches=1000, device='cuda:0', save_dir=None):
    """
    Iterates through the pile_dedup_sample dataset and compiles a new all_tokens tensor.
    If a saved tensor exists at save_dir, loads that instead of recomputing.

    Args:
        sequence_length (int): The length of each token sequence.
        batch_size (int): The number of sequences per batch.
        max_batches (int, optional): The maximum number of batches to process. Defaults to None.
        device (str): The device to store the tensor on.
        save_dir (str, optional): Directory to save/load tensor from. If None, uses default name.

    Returns:
        torch.Tensor: A tensor containing all tokenized sequences.
    """
    # Create default save path if none provided
    if save_dir is None:
        save_dir = f"all_tokens_seq{sequence_length}_batch{batch_size}_max{max_batches}.pt"
    save_path = Path(save_dir)

    # Check if tensor already exists
    if save_path.exists():
        print(f"Loading existing tensor from {save_path}")
        return torch.load(save_path).to(device)

    # Load dataset
    pile_dedup_sample = load_dataset('EleutherAI/the_pile_deduplicated', streaming=True, split='train')

    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained('EleutherAI/pythia-160m-deduped')

    all_token_batches = []

    current_batch = []
    batch_count = 0

    print("Compiling all_tokens tensor...")
    pbar = tqdm(total=max_batches)
    for sample in pile_dedup_sample:
        # Tokenize the text
        tokens = tokenizer.encode(sample['text'])
        print(tokens[:10])
        # Get as many sequence_length chunks as possible from tokens
        for i in range(0, len(tokens) - sequence_length + 1, sequence_length):
            current_batch.append(tokens[i:i + sequence_length])
            
            # If batch is full, add to all_token_batches
            if len(current_batch) == batch_size:
                batch_tensor = torch.tensor(current_batch, dtype=torch.int32)
                all_token_batches.append(batch_tensor)
                current_batch = []
                batch_count += 1
                pbar.update(1)
                # Check if we've reached the maximum number of batches
                if max_batches and batch_count >= max_batches:
                    break
        
        # Break outer loop if we've hit max_batches
        if max_batches and batch_count >= max_batches:
            break

    pbar.close()

    # Handle the last batch if it's not empty
    if current_batch:
        batch_tensor = torch.tensor(current_batch, dtype=torch.int32)
        all_token_batches.append(batch_tensor)

    # Concatenate all batches into a single tensor
    all_tokens = torch.cat(all_token_batches, dim=0).to(device)

    # Save tensor
    print(f"Saving tensor to {save_path}")
    save_path.parent.mkdir(parents=True, exist_ok=True)
    #torch.save(all_tokens.cpu(), save_path)

    print(f"Compiled all_tokens tensor with shape: {all_tokens.shape}")
    return all_tokens

In [15]:
a = compile_all_tokens2(max_batches=200)

Resolving data files:   0%|          | 0/1650 [00:00<?, ?it/s]

Compiling all_tokens tensor...


  0%|          | 0/200 [00:00<?, ?it/s]

[1147, 310, 2218, 13, 285, 9262, 15, 1422, 476, 1132]
[46167, 27, 9838, 2254, 21811, 33584, 187, 187, 38859, 6138]
[10572, 8481, 322, 5247, 401, 3647, 3976, 1336, 187, 187]
[32197, 10168, 13, 10396, 1294, 4694, 285, 253, 29242, 366]
[36751, 5464, 29961, 7885, 19544, 13, 23011, 13, 3382, 13372]
[92, 187, 50276, 3, 71, 793, 24314, 1381, 3925, 13]
[31484, 187, 187, 9710, 187, 187, 37, 5338, 342, 952]
[75, 4985, 50, 327, 380, 4145, 46, 6807, 32, 187]
[42, 1849, 6311, 253, 14164, 5809, 14013, 908, 275, 6700]
[15271, 8808, 254, 9491, 187, 187, 15271, 8808, 254, 9491]
[49579, 896, 247, 4804, 7663, 187, 187, 2866, 1466, 479]
[10457, 619, 10438, 13, 309, 3597, 281, 9580, 347, 1199]
[18681, 15, 29580, 16732, 313, 42, 14, 28889, 10, 588]
[49, 4741, 27, 8966, 4632, 8698, 559, 247, 8461, 187]
[510, 35247, 6480, 14, 15505, 390, 25115, 9606, 407, 22085]
[12547, 27, 3599, 899, 12000, 8585, 187, 187, 25494, 1982]
[50273, 510, 14568, 3927, 273, 253, 13338, 2111, 273, 9892]
[510, 767, 5971, 2634, 44, 110

  4%|▍         | 8/200 [00:05<01:43,  1.86it/s]

[187, 187, 15611, 9643, 187, 187, 4, 40604, 187, 187]
[22396, 187, 187, 510, 8215, 310, 247, 3569, 985, 342]
[7142, 3979, 23249, 27, 36388, 2716, 273, 512, 32846, 342]
[33, 13982, 370, 8695, 33725, 27, 11188, 6905, 13, 87]
[2697, 884, 15, 19, 428, 367, 8592, 1574, 37193, 16530]
[510, 270, 386, 312, 6712, 16928, 273, 399, 35447, 13]
[283, 48934, 422, 187, 283, 15684, 1214, 46998, 16, 15905]
[18091, 272, 187, 187, 14102, 71, 15390, 3959, 247, 2491]
[2347, 4031, 4448, 3599, 5830, 434, 686, 45, 16580, 8]
[29, 15597, 2730, 22225, 31, 187, 29, 53, 43561, 31]
[510, 3522, 310, 3809, 17682, 323, 776, 7203, 18240, 15]
[2044, 3596, 426, 551, 187, 50274, 881, 27, 1777, 12290]
[3220, 22885, 9420, 571, 18605, 374, 14, 33590, 1283, 187]
[16395, 456, 327, 253, 611, 832, 267, 9409, 8451, 275]
[55, 1079, 372, 418, 5738, 1108, 16363, 29700, 327, 7565]
[25810, 14, 15288, 266, 8246, 187, 187, 688, 1340, 281]
[510, 3166, 297, 28598, 313, 12731, 10, 187, 187, 22117]
[2374, 4247, 337, 13, 4765, 13, 253, 954, 

  6%|▌         | 12/200 [00:07<01:34,  1.99it/s]

[1992, 3048, 281, 253, 2862, 1789, 13, 15596, 436, 3048]
[510, 40602, 326, 2186, 253, 33852, 4430, 327, 403, 1638]
[605, 187, 605, 50276, 40, 11927, 1658, 3364, 31483, 5443]
[510, 7899, 6120, 476, 320, 247, 6422, 12215, 323, 253]
[4505, 15, 5758, 3914, 92, 18, 27, 3967, 64, 936]
[1845, 14843, 13, 26281, 13, 3522, 285, 4948, 323, 416]
[9709, 652, 29151, 24466, 27, 380, 15300, 54, 43667, 13]
[8207, 9038, 407, 2451, 1616, 1880, 253, 6167, 4991, 310]
[22, 15, 6049, 858, 2305, 15, 19498, 45035, 3599, 249]
[18968, 29012, 1720, 30345, 14307, 347, 1457, 2434, 14, 21689]
[5, 25410, 5731, 187, 5, 25410, 3334, 1121, 312, 187]
[16, 69, 1641, 14, 87, 18, 16, 28, 187, 187]
[26788, 13, 3285, 4223, 4332, 187, 187, 37701, 1658, 7736]
[688, 14, 14996, 3492, 12089, 310, 253, 747, 2806, 15]
[34, 4804, 1659, 281, 6431, 6849, 13, 642, 2647, 849]
[34, 5579, 10651, 13, 2305, 18456, 18847, 285, 247, 3513]
[28, 187, 28, 23709, 1873, 273, 6246, 32857, 24227, 52]
[35947, 29019, 273, 11271, 12113, 13, 20859, 38793

KeyboardInterrupt: 

  6%|▋         | 13/200 [00:18<01:34,  1.99it/s]

In [8]:
default_cfg = {
    "seed": 49,
    "batch_size": 128,
    "buffer_mult": 128,
    "lr": 5e-5,
    "num_tokens": 400_000_000,
    "l1_coeff": 2,
    "beta1": 0.9,
    "beta2": 0.999,
    "d_in": base_model.cfg.d_model,
    "dict_size": 768*8*2,
    "seq_len": 256,
    "enc_dtype": "fp32",
    "model_name": "pythia-160m-deduped",
    "site": "resid_pre",
    "device": "cuda:0",
    "model_batch_size": 4,
    "log_every": 100,
    "save_every": 30000,
    "dec_init_norm": 0.08,
    "hook_point": "blocks.5.hook_resid_pre",
    "wandb_project": "crosscoder-fun",
    "wandb_run_name": "toy-run-0",
}
cfg = arg_parse_update_cfg(default_cfg)

In IPython - skipped argparse


In [9]:
%load_ext autoreload
%autoreload 2
from buffer import Buffer
newb  = Buffer(cfg, base_model, checkpoint_mid_model, actual_toks[:128])


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
torch.Size([4, 256, 768])


AssertionError: 

In [5]:
256*128*1000

32768000

In [4]:
default_cfg = {
    "seed": 49,
    "batch_size": 128,
    "buffer_mult": 128,
    "lr": 5e-5,
    "num_tokens": 400_000_000,
    "l1_coeff": 2,
    "beta1": 0.9,
    "beta2": 0.999,
    "d_in": base_model.cfg.d_model,
    "dict_size": 768*8*2,
    "seq_len": 256,
    "enc_dtype": "fp32",
    "model_name": "pythia-160m-deduped",
    "site": "resid_pre",
    "device": "cuda:0",
    "model_batch_size": 4,
    "log_every": 100,
    "save_every": 30000,
    "dec_init_norm": 0.08,
    "hook_point": "blocks.5.hook_resid_pre",
    "wandb_project": "crosscoder-fun",
    "wandb_run_name": "toy-run-0",
}
cfg = arg_parse_update_cfg(default_cfg)

trainer = Trainer(cfg, base_model, checkpoint_mid_model, actual_toks)
trainer.train()
# %%

In IPython - skipped argparse


Estimating norm scaling factor: 100%|██████████| 100/100 [00:03<00:00, 25.01it/s]


Refreshing the buffer!


100%|██████████| 16/16 [00:01<00:00, 11.92it/s]
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tim_hua. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


  0%|          | 10/3125000 [00:00<17:31:17, 49.54it/s]

...

Refreshing the buffer!


  0%|          | 4403/3125000 [01:20<14:48:29, 58.54it/s]

{'loss': 72.37870788574219, 'l2_loss': 17.371700286865234, 'l1_loss': 976.6868896484375, 'l0_loss': 6639.3984375, 'l1_coeff': 0.05632, 'lr': 5e-05, 'explained_variance': 0.9848951101303101, 'explained_variance_A': 0.983425498008728, 'explained_variance_B': 0.9847656488418579}
Refreshing the buffer!


  0%|          | 4409/3125000 [01:20<15:52:50, 54.58it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/workspace/crosscoder-model-diff-replication/checkpoints'

In [4]:
?load_dataset

Signature:
load_dataset(
    path: str,
    name: Optional[str] = None,
    data_dir: Optional[str] = None,
    data_files: Union[str, Sequence[str], Mapping[str, Union[str, Sequence[str]]], NoneType] = None,
    split: Union[str, datasets.splits.Split, NoneType] = None,
    cache_dir: Optional[str] = None,
    features: Optional[datasets.features.features.Features] = None,
    download_config: Optional[datasets.download.download_config.DownloadConfig] = None,
    download_mode: Union[datasets.download.download_manager.DownloadMode, str, NoneType] = None,
    verification_mode: Union[datasets.utils.info_utils.VerificationMode, str, NoneType] = None,
    keep_in_memory: Optional[bool] = None,
    save_infos: bool = False,
    revision: Union[str, datasets.utils.version.Version, NoneType] = None,
    token: Union[bool, str, NoneType] = None,
    streaming: bool = False,
    num_proc: Optional[int] = None,
    storage_options: Optional[Dict] = None,
    trust_remote_code: bool = None,
   